# Direktna metoda izračuna astronomskega položaja

Sledijo komentarji

In [ ]:
import os, sys

# add custom modules and astro data path 
pp = '../nav_tools/'
sys.path.append(pp)

In [ ]:
import math as mat
import numpy as np
import matplotlib.pyplot as mpl
mpl.rcParams['text.usetex'] = True
mpl.rcParams.update({'font.size': 7})

import celestialdata as cdata
import navigationalstars as ns
import navtools as nt

In [ ]:
# Returns Geographic Position (GP) of the star based on DEC and GHA
# input: declination, GHA
# output: fi, lambda

def getZFL(h, dec, gha):
    
    z = mat.pi/2 - h
    fi = dec
    if gha > mat.pi:
        la = 2*mat.pi - gha
    else:
        la = -gha
    
    return [z,fi,la]

In [ ]:
# Returns great circle (GC) distance between two positions
# - input: fi1, fi2, dl (delta lambda)
# - output: great circle distance in radians

def getDist(fi1,fi2,dl):
    
    psi1 = mat.pi/2 - fi1
    psi2 = mat.pi/2 - fi2
    
    cosD = mat.cos(psi1)*mat.cos(psi2) + mat.sin(psi1)*mat.sin(psi2)*mat.cos(dl)
    
    return mat.acos(cosD)

In [ ]:
#

def getPi1(z1,z2,D):
    
    cosPi1 = (mat.cos(z2) - mat.cos(z1)*mat.cos(D)) / (mat.sin(z1)*mat.sin(D))
    
    return mat.acos(cosPi1)

In [ ]:
#

def getW1(fi1,fi2,D):
    
    psi1 = mat.pi/2 - fi1
    psi2 = mat.pi/2 - fi2
    
    cosW1 = (mat.cos(psi2) - mat.cos(psi1)*mat.cos(D)) / (mat.sin(psi1)*mat.sin(D))
    
    return mat.acos(cosW1)

In [ ]:
#

def getPsiX(z1, fi1, Wx):
    
    psi1 = mat.pi/2 - fi1
    
    cosPsiX = mat.cos(psi1)*mat.cos(z1) + mat.sin(psi1)*mat.sin(z1)*mat.cos(Wx)
    
    return mat.acos(cosPsiX)

In [ ]:
#

def getDLx(z1, fi1, fix):
    
    psi1 = mat.pi/2 - fi1
    psix = mat.pi/2 - fix
    
    cosDL = (mat.cos(z1) - mat.cos(psi1)*mat.cos(psix)) / (mat.sin(psi1)*mat.sin(psix))
    
    return mat.acos(cosDL)

In [ ]:
#

def calculatePosition(z1,fi1,la1,z2,fi2,la2):
    
    # delta fi and delta lambda
    dfi = fi2 - fi1
    dla = la2 - la1
    if dla > mat.pi:
        dla = -(2*mat.pi - dla)
    elif dla < -mat.pi:
        dla = 2*mat.pi + dla

    #print('dla={:}'.format(rad2deg(dla)))

    D = getDist(fi1,fi2, dla)
    Pi1 = getPi1(z1,z2,D)
    W1 = getW1(fi1,fi2,D)
    alpha1 = W1 - Pi1
    print('D={:}, W1={:}, Pi1={:}, a1={:}'.format(nt.rad2deg(D), nt.rad2deg(W1), nt.rad2deg(Pi1), nt.rad2deg(alpha1)))

    psiX = getPsiX(z1,fi1,alpha1)
    fiX = mat.pi/2 - psiX
    dlX = getDLx(z1,fi1,fiX)
    print('psiX={:}, dlX={:}'.format(nt.rad2deg(psiX), nt.rad2deg(dlX)))
    
    if dla < 0:
        laX = la1 - np.abs(dlX)
    else:
        laX = la1 + np.abs(dlX)
    
    return [fiX, laX]

In [ ]:
# observer position (what should we get)!!
fi_check = [46,0,'N'];  fi_check_rad = nt.deg2rad(nt.nav2dd(fi_check))
la_check = [13,30,'E']; la_check_rad = nt.deg2rad(nt.nav2dd(la_check))

# Inputs: Star 1
h1 = [31,02.2]; h1_rad = nt.deg2rad(nt.dms2dd(h1))
dec1 = [7,24.7]; dec1_rad = nt.deg2rad(nt.dms2dd(dec1))
gha1 = [294,21.7]; gha1_rad = nt.deg2rad(nt.dms2dd(gha1))

[z1,fi1,la1] = getZFL(h1_rad,dec1_rad,gha1_rad)
print('*Star 1* - GP and zenith distance:')
print('  -> fi =  {:}'.format(nt.prettyPrintLat(nt.rad2deg(fi1))))
print('  -> la = {:}'.format(nt.prettyPrintLong(nt.rad2deg(la1))))
print('  ->  z =  {:} (observed)'.format(nt.prettyPrintAlt(nt.rad2deg(z1))))
print()

# Inputs: Star 2
h2 = [67,13.9]; h2_rad = nt.deg2rad(nt.dms2dd(h2))
dec2 = [23,34.3]; dec2_rad = nt.deg2rad(nt.dms2dd(dec2))
gha2 = [351,21.0]; gha2_rad = nt.deg2rad(nt.dms2dd(gha2))

[z2,fi2,la2] = getZFL(h2_rad,dec2_rad,gha2_rad)
print('*Star 2* - GP and zenith distance:')
print('  -> fi =  {:}'.format(nt.prettyPrintLat(nt.rad2deg(fi2))))
print('  -> la = {:}'.format(nt.prettyPrintLong(nt.rad2deg(la2))))
print('  ->  z =  {:} (observed)'.format(nt.prettyPrintAlt(nt.rad2deg(z2))))
print()

print('-----------------------------------------')
print('Calculation parameters:')
[fiX, laX] = calculatePosition(z1,fi1,la1,z2,fi2,la2)
print('-----------------------------------------')
print()

print('*** Calculated position: ***')
print('  -> fi =  {:}'.format(nt.prettyPrintLat(nt.rad2deg(fiX))))
print('  -> la = {:}'.format(nt.prettyPrintLong(nt.rad2deg(laX))))
print()
print('Error in calculated position')
print('  -> fi = {:.3f} Nm'.format(np.abs(fi_check_rad-fiX)/mat.pi*180*60))
print('  -> la = {:.3f} Nm'.format(np.abs(la_check_rad-laX)/mat.pi*180*60))
